# Indicadores de Salud

Cálculo de las principales enfermedades crónicas en población de la Ciudad de México a partir de la Encuesta Nacional de Salud y Nutrición

In [135]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, srvyr, janitor, stringr, haven, writexl)

Tabla de resultados y filtrarlo a nivel Ciudad de México

In [139]:
# Se usa read_csv2 para leer el archivo csv con separador de punto y coma
sangre <- read_csv2("ensasangre21_entrega_w_integrada_ok.csv", show_col_types = FALSE) %>%
clean_names() %>%
mutate(across(where(is.character), as.numeric)) %>%
filter(entidad == 9)

antropo <- read_csv2("ensaantro21_entrega_w_17_12_2021.csv", show_col_types = FALSE) %>%
clean_names() %>%
mutate(across(where(is.character), as.numeric)) %>%
filter(entidad == 9)


ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"There were 31 warnings in `mutate()`.
The first warning was:
ℹ In argument: `across(where(is.character), as.numeric)`.
Caused by warning:
! NAs introducidos por coerción
ℹ Run `dplyr::last_dplyr_warnings()` to see the 30 remaining warnings."
ℹ Using "','" as decimal and "'.'" as grouping mark. Use `read_delim()` for more control.

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Warning message:
"There were 17 warnings in `mutate()`.
The first warning was:
ℹ In argument: `across(where(is.character), as.numeric)`.
Caused by warning:
! NAs introducidos por coerción
ℹ Run `dplyr::last_dplyr_warnings()` to see the 16 remaining warnings."


Población con Diabetes

Criterios para obtenidos de: *Guía de Práctica Clínica: Diagnóstico y Tratamiento de Diabetes Mellitus Tipo 2. Instituto Mexicano del Seguro Social, (2018)*

-Glucemia plasmática en ayuno: >= 125 mg/dL

-Glucemia plasmática después de dos horas del último alimento: >= 200 mg/dL

-Hemoglobina glucosificada: >= 6.5%



In [140]:
pob_diab <- sangre %>%
select(san04, h0302, h0303, a0301, valor_glu_suero, valor_hb1ac, ponde_g) %>%
rename(ult_alim = san04, sexo = h0302, edad = h0303, diag_diab = a0301, glucosa = valor_glu_suero, hemoglobina = valor_hb1ac) %>%
filter(complete.cases(.)) %>%
summarise(
    pob_total = sum(ifelse(edad >= 20, ponde_g, 0)),
    pob_diab = sum(
        ifelse(edad >= 20 & ult_alim >= 8 & glucosa >= 125, ponde_g, 0),
        ifelse(edad >= 20 & ult_alim >= 2 & glucosa >= 200, ponde_g, 0), 
        ifelse(hemoglobina >= 6.5, ponde_g, 0)),
    porcent_pob = pob_diab / pob_total * 100 )
    

head(pob_diab)

pob_total,pob_diab,porcent_pob
<dbl>,<dbl>,<dbl>
6998165,2244801,32.077


Población con Obesidad (Revisar los resultados)

Criterios para obtenidos de *Guía de Práctica Clínica: Intervención dietética: Paciente con Obesidad. Instituto Mexicano del Seguro Social, (2013)*

Índice de Masa Corporal: Peso (kg)/ (Altura (m))^2

-Grado 1: IMC 30-34.9
-Grado 2: IMC 35 – 39.9
-Grado 3: IMC 40 – 49
-Grado 4: IMC ≥50

In [147]:
pob_obe <- antropo %>%
select(h0302, h0303, an01_1, an01_2, an04_1, an04_2, ponde_f) %>%
rename(sexo = h0302, edad = h0303, peso1 = an01_1, peso2 = an01_2, altura1 = an04_1, altura2 = an04_2) %>%
filter(complete.cases(.)) %>%
summarise(
    pob_total = sum(ifelse(edad >= 20, ponde_f, 0)),
    pob_obe = sum(ifelse(edad >= 20 & (peso1 / (altura1 / 100)^2) >= 30, ponde_f, 0),
    ifelse(edad >= 20 & (peso2 / (altura2 / 100)^2) >= 30, ponde_f, 0)),
    porcent_pob = pob_obe / pob_total * 100 )

pob_obe


pob_total,pob_obe,porcent_pob
<dbl>,<dbl>,<dbl>
5409125,4261477,78.78311


Pendiente Población con Anemia

Criterios para obtenidos de *Guía de Práctica Clínica: Prevención, Diagnóstico y Tratamiento de la Anemia por Deficiencia de Hierro. Instituto Mexicano del Seguro Social, (2010)*